# Extracting data from csvs

In [2]:
%store -r sensorIds
print(sensorIds)

['18699', '18720', '18749']


In [3]:
#temporary reassigning of paths (since plume dashboard not working)

# we need two dictionaries (one for measurements and one for location)
measure_dictionary = {}
location_dictionary = {}

sensorPaths = []

for snum in sensorIds:
    sensorPaths.append("..\\data\\flow\\temp\\sensor_"  + snum)
print(sensorPaths)


['..\\data\\flow\\temp\\sensor_18699', '..\\data\\flow\\temp\\sensor_18720', '..\\data\\flow\\temp\\sensor_18749']


In [4]:
import pandas as pd
import numpy as np
import psycopg2
from datetime import datetime
from datetime import timezone
import json
import os

In [10]:
csvMeasurementsPath = []
csvPostionsPath = []

for i in sensorPaths:
    with os.scandir(i) as listOfFiles:
        for currentFile in listOfFiles:
            # get all files that are csv
            if currentFile.is_file() and currentFile.name.endswith('csv'):
                 if "positions" not in currentFile.name:
                    csvMeasurementsPath.append(os.path.join(i,currentFile.name)) 
                 else:
                    csvPostionsPath.append(os.path.join(i,currentFile.name)) 

In [13]:
#map sensor id to filepaths
measurement_dictionary = {k:v for k,v in zip(sensorIds,csvMeasurementsPath)}
location_dictionary = {k:v for k,v in zip(sensorIds,csvPostionsPath)}
print(measurement_dictionary)
print(location_dictionary)

{'18699': '..\\data\\flow\\temp\\sensor_18699\\user_measures.csv', '18720': '..\\data\\flow\\temp\\sensor_18720\\user_measures.csv', '18749': '..\\data\\flow\\temp\\sensor_18749\\user_measures.csv'}
{'18699': '..\\data\\flow\\temp\\sensor_18699\\user_positions.csv', '18720': '..\\data\\flow\\temp\\sensor_18720\\user_positions.csv', '18749': '..\\data\\flow\\temp\\sensor_18749\\user_positions.csv'}


# ....

In [14]:
#bringing the csv to pandas dataframes
#put the dataframes into a dictionary with the sensor id as the key
for key in sensorIds:
    locations = {} #intialise empty dictionary to store each day of locations
    
    df_temp = pd.read_csv(location_dictionary[key],parse_dates=True, index_col="timestamp")
    
    # try using the dates which are already supplied. 
    # This strategy in the line below converts them and rounds down to date using 'd' flag
    # df_temp['day'] = pd.to_datetime(df_temp['date'], errors='coerce').dt.floor('d')
    # However, they are still datetimes so there is still a time - it's just been rounded down to zero, 
    # e.g. '2021-07-17T00:00:00.000000000'
    # This strategy (line below) really will keep just the date
    df_temp['day'] = pd.to_datetime(df_temp['date'], errors='coerce').dt.date
    
    print(df_temp['day'])
    
    the_unique_dates = df_temp['day'].unique()
    #print('Unique dates:',the_unique_dates)
    # this gives the same result as the for loop below
  
    #splitting the dataframe into separate days
    #for each day in unique dates set:
    for day in the_unique_dates:
        try:
            # In my code below I assign the subset of records to a new dataframe called dft
            # create 'midnight' timestamps
            st = pd.to_datetime(day, errors='coerce')
            # add one day to this midnight
            et = st + pd.DateOffset(1)
            # cast both to integers to make neat keys
            sts = int(st.timestamp())
            ets = int(et.timestamp())
            #print(sts, ets)

            # concatenating numbers into text: 
            timestampKey = "%d_%d" % (sts, ets)
                
            # select the records for this day
            dft = df_temp[df_temp['day']==day]
            locations[timestampKey] = dft

        except (KeyError):
            print(KeyError)
    #assign new value to the locations dictionary, so it is associated in key-value pair with its coressponding sensorid
    location_dictionary[key] = locations
    #print(df_temp['day'])


timestamp
1626501291   2021-07-17 05:54:00
1626502670   2021-07-17 06:17:00
1626502734   2021-07-17 06:18:00
1626503572   2021-07-17 06:32:00
1626503578   2021-07-17 06:32:00
                     ...        
1628086932   2021-04-08 14:22:00
1628086964   2021-04-08 14:22:00
1628086995   2021-04-08 14:23:00
1628087027   2021-04-08 14:23:00
1628087058   2021-04-08 14:24:00
Name: day, Length: 5139, dtype: datetime64[ns]
timestamp
1626159288   2021-07-13 06:54:00
1626159340   2021-07-13 06:55:00
1626159370   2021-07-13 06:56:00
1626159548   2021-07-13 06:59:00
1626159592   2021-07-13 06:59:00
                     ...        
1628082493   2021-04-08 13:08:00
1628085175   2021-04-08 13:52:00
1628085710   2021-04-08 14:01:00
1628086090   2021-04-08 14:08:00
1628087025   2021-04-08 14:23:00
Name: day, Length: 449, dtype: datetime64[ns]
timestamp
1626448226   2021-07-16 15:10:00
1626448246   2021-07-16 15:10:00
1626448276   2021-07-16 15:11:00
1626450376   2021-07-16 15:46:00
1626452845   2021-0

In [14]:
total = 0
for key in location_dictionary:
    for timestampKey in location_dictionary[key]:
    #   print(timestampKey)
    # break
      total += len(location_dictionary[key][timestampKey].index.values)
    print("sensor: "  + str(key) + "_total:" + str(total))
    total = 0 #reset the total

print("\n")

test = {k:v for k,v in zip(sensorIds,csvPostionsPath)}
for key in sensorIds:
    df_temp = pd.read_csv(test[key],parse_dates=True, index_col="timestamp")
    print("sensor: "  + str(key) + "_total:" + str(len(df_temp.index.values)))


sensor: 18699_total:5139
sensor: 18720_total:449
sensor: 18749_total:1113


sensor: 18699_total:5139
sensor: 18720_total:449
sensor: 18749_total:1113


In [14]:
#bringing the csv to pandas dataframes
#put the dataframes into a dictionary with the sensor id as the key
for key in sensorIds:

    measurements = {} #intialse empty dictioanry to store each day of measurements

    df_temp = pd.read_csv(measurement_dictionary[key],parse_dates=True, index_col="timestamp")


    #make a new day column to store the datetime.date of each record
    df_temp['day'] = pd.to_datetime(df_temp['date (UTC)'], errors='coerce').dt.date
   
   
    the_unique_dates = df_temp['day'].unique()

    #splitting the dataframe into separate days
    #for each day in unique dates set:
    for day in the_unique_dates:
        try:
            # In my code below I assign the subset of records to a new dataframe called dft
            # create 'midnight' timestamps
            st = pd.to_datetime(day, errors='coerce')
            # add one day to this midnight
            et = st + pd.DateOffset(1)
            # cast both to integers to make neat keys
            sts = int(st.timestamp())
            ets = int(et.timestamp())
            #print(sts, ets)

            # concatenating numbers into text: 
            timestampKey = "%d_%d" % (sts, ets)
                
            # select the records for this day
            dft = df_temp[df_temp['day']==day]
            measurements[timestampKey] = dft

        except (KeyError):
            print("...")

    measurement_dictionary[key] = measurements #clear dataframe
    
    #print(df_temp['day'])

    # df = measure_dictionary[key]
    # df.drop("date (UTC)", axis=1, inplace=True)



timestamp
1.626501e+09    2021-07-17
1.626501e+09    2021-07-17
1.626501e+09    2021-07-17
1.626501e+09    2021-07-17
1.626502e+09    2021-07-17
                   ...    
1.628083e+09    2021-08-04
1.628083e+09    2021-08-04
1.628083e+09    2021-08-04
1.628084e+09    2021-08-04
1.628084e+09    2021-08-04
Name: day, Length: 25507, dtype: object
timestamp
1.625901e+09    2021-07-10
1.625901e+09    2021-07-10
1.625901e+09    2021-07-10
1.625901e+09    2021-07-10
1.625901e+09    2021-07-10
                   ...    
1.628083e+09    2021-08-04
1.628084e+09    2021-08-04
1.628084e+09    2021-08-04
1.628084e+09    2021-08-04
1.628084e+09    2021-08-04
Name: day, Length: 16520, dtype: object
timestamp
1.626448e+09    2021-07-16
1.626448e+09    2021-07-16
1.626448e+09    2021-07-16
1.626448e+09    2021-07-16
1.626448e+09    2021-07-16
                   ...    
1.628081e+09    2021-08-04
1.628081e+09    2021-08-04
1.628081e+09    2021-08-04
1.628081e+09    2021-08-04
1.628081e+09    2021-08-04

In [13]:
total = 0
for key in measurement_dictionary:
    for timestampKey in measurement_dictionary[key]:
    #     print(timestampKey)
    # break       
        total += len(measurement_dictionary[key][timestampKey].index.values)
    print("sensor: "  + str(key) + "_total:" + str(total))
    total = 0 #reset the total

print("\n")

test = {k:v for k,v in zip(sensorIds,csvMeasurementsPath)}
for key in sensorIds:
    df_temp = pd.read_csv(test[key],parse_dates=True, index_col="timestamp")
    print("sensor: "  + str(key) + "_total:" + str(len(df_temp.index.values)))

sensor: 18699_total:25507
sensor: 18720_total:16520
sensor: 18749_total:27043


sensor: 18699_total:25507
sensor: 18720_total:16520
sensor: 18749_total:27043


In [ ]:
measurement_dictionary[sensorIds[0]]

In [9]:
for key in sensorIds:
    for l_dictionary in location_dictionary[key]:
        for timestampKey in l_dictionary:
            df = l_dictionary[timestampKey]
            print(df.head(1))
        
# min_long= df['longitude'].min()
# max_long = df['longitude'].max()

# min_lat= df['latitude'].min()
# max_lat = df['latitude'].max()

# print("min_longitude: " + str(min_long) + "\n max_logitude: " + str(max_long))
# print("min_latitude: " + str(min_lat) + "\n max_latitude: " + str(max_lat))

# Preparing data for upload into PostGres

In [12]:
sensor_summaries = {}
sensor_data = {}

geometry_string = ""

ldf = df
mdf = df

for timestampKey in location_dictionary[sensorIds[0]]:
    #location dataframe
    ldf = locations[timestampKey]

    #create bounding box polygon
    min_y= ldf['longitude'].min()
    max_y = ldf['longitude'].max()

    min_x= ldf['latitude'].min()
    max_x = ldf['latitude'].max()
    #POLYGON(minx miny, minx Maxy, maxx Maxy, maxx miny, minx miny)
    geometry_string = "POLYGON(({} {}, {} {}, {} {}, {} {},{} {}))".format(min_x,min_y,   min_x,max_y,   max_x,max_y,   max_x,min_y,   min_x,min_y)

print(geometry_string)
        
    # for timestampKey in measure_dictionary[key]:
        
    #     #create new key for sensor summaries
    #     timestamp_sensor_key = str(df.index[0]) + "_" + str(df.index[-1]) + "_" +  str(int(key))

    #     #measurement dataframe
    #     mdf = measurements[timestampKey]

    #     #summaryArray = [timestamp_start,timestamp_end,sensor_id,bouding_box,measurement_count]
    #     summaryArray = [mdf.index[0],mdf.index[-1],int(key),geometry_string,len(mdf.index.values)] #inserting row into temp array
    #     sensor_summaries[timestamp_sensor_key] = summaryArray    #assign new dataframe to coressponding key
        
    #     ##########print(sensor_summaries[timestamp_sensor_key])

    #     #dataArray = [id, mesaurement_json,location_json]
    #     dataArray = [mdf.to_json(orient="columns"),ldf.to_json(orient="columns")]     #dataArray = [mdf.to_json('./{}.json'.format(key),orient="columns"),ldf.to_json(orient="columns")]
    #     sensor_data[timestamp_sensor_key] = dataArray    #assign new dataframe to coressponding key    

In [10]:
# min_long= df['longitude'].min()
# max_long = df['longitude'].max()

# min_lat= df['latitude'].min()
# max_lat = df['latitude'].max()

# print("min_longitude: " + str(min_long) + "\n max_logitude: " + str(max_long))
# print("min_latitude: " + str(min_lat) + "\n max_latitude: " + str(max_lat))

In [11]:
# sensor_summaries = {}
# sensor_data = {}

# geometry_string = ""

# ldf = df
# mdf = df

# for key in sensorIds:
#     for timestampKey in location_dictionary[sensorIds[0]]:
#         #location dataframe
#         ldf = locations[timestampKey]

#         #create bounding box polygon
#         min_y= ldf['longitude'].min()
#         max_y = ldf['longitude'].max()

#         min_x= ldf['latitude'].min()
#         max_x = ldf['latitude'].max()
#         #POLYGON(minx miny, minx Maxy, maxx Maxy, maxx miny, minx miny)
#         geometry_string = "POLYGON(({} {}, {} {}, {} {}, {} {},{} {}))".format(min_x,min_y,   min_x,max_y,   max_x,max_y,   max_x,min_y,   min_x,min_y)

# print(geometry_string)
        
#     # for timestampKey in measure_dictionary[key]:
        
#     #     #create new key for sensor summaries
#     #     timestamp_sensor_key = str(df.index[0]) + "_" + str(df.index[-1]) + "_" +  str(int(key))

#     #     #measurement dataframe
#     #     mdf = measurements[timestampKey]

#     #     #summaryArray = [timestamp_start,timestamp_end,sensor_id,bouding_box,measurement_count]
#     #     summaryArray = [mdf.index[0],mdf.index[-1],int(key),geometry_string,len(mdf.index.values)] #inserting row into temp array
#     #     sensor_summaries[timestamp_sensor_key] = summaryArray    #assign new dataframe to coressponding key
        
#     #     ##########print(sensor_summaries[timestamp_sensor_key])

#     #     #dataArray = [id, mesaurement_json,location_json]
#     #     dataArray = [mdf.to_json(orient="columns"),ldf.to_json(orient="columns")]     #dataArray = [mdf.to_json('./{}.json'.format(key),orient="columns"),ldf.to_json(orient="columns")]
#     #     sensor_data[timestamp_sensor_key] = dataArray    #assign new dataframe to coressponding key    

In [11]:
# # reading the JSON data using json.loads(json string)
# # converting json dataset from dictionary to dataframe
# dict_data = json.loads(sensor_data[sensorIds[0]][0])
# data = pd.DataFrame.from_dict(dict_data, orient='columns')
# data.index.rename('timestamp', inplace=True)
# data.head()

In [21]:
for key in sensor_summaries:
    mdf = sensor_summaries[key]
    print(mdf)


[1626480043.0, 1626566444.0, 18699, '', 1434]
[1626480043.0, 1626566444.0, 18720, '', 1434]
[1626480043.0, 1626566444.0, 18749, '', 1434]


# Exporting to PostgresSQL 


# Writing records from a Dictionary of arrays to a SQL database
loop over all the keys and execute insert query

In [12]:
#Connecting to an existing database
sdgshehe
con = psycopg2.connect(
    host="localhost",
    database="airQuality",
    user="Riyad", 
    password="123",
    # attempt to connect for 3 seconds then raise exception
    connect_timeout = 3)

In [13]:
#Opening a cursor to execute database operations
cursor = con.cursor()
query = "SELECT * FROM sensor_network.sensors"
#change dataframe to csv and save file
sensorsdf = pd.read_sql_query(query, con, index_col='plume_id')

cursor.close()

In [14]:
sensorsdf.head()

,sensor_serial_number,id,type_id
plume_id,,,
17539,02:00:00:00:43:8b,1,1
19115,02:00:00:00:49:b3,2,1
17521,02:00:00:00:43:79,3,1
16397,02:00:00:00:3f:15,4,1
16701,02:00:00:00:40:45,5,1


In [15]:
for key in sensorIds:
    #get the new key from sensors table
    sensor_id = sensorsdf.loc[int(key)][1]

    mdf = sensor_summaries[key]
    ldf = sensor_data[key]

    try:
        #Opening a cursor to execute database operations
        cursor = con.cursor()

        #inserting sensor data and return the id of new record
        cursor.execute("INSERT INTO sensor_data.archive_measurements (measurements,locations) VALUES(%s, %s) \n RETURNING id", (str(ldf[0]),str(ldf[1])) )
        con.commit() 
        
        #set id of new record into local variable
        sensor_data_id = cursor.fetchone()[0]

        #inserting sensor summary
        cursor.execute("INSERT INTO sensor_network.sensor_summaries (timestamp_start,timestamp_end, sensor_id, b_box, sensor_data_id, measurement_count) VALUES(%s, %s, %s, %s, %s, %s)", (int(mdf[0]), int(mdf[1]), int(sensor_id),str(mdf[3]),int(sensor_data_id),int(mdf[4])))
        con.commit() 

        cursor.close()
    #if table name does not exist exit loop 
    except(psycopg2.errors.UndefinedTable) as error:
        print('ERROR: ' + error)
        break

In [16]:
#closing the connection
con.close()